In [ ]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import os
%matplotlib inline
from scipy.cluster.vq import kmeans
from scipy.cluster.vq import vq


In [ ]:

# Load the image
image1 = cv2.imread('./data/images/IMG_20241223_150037.jpg')
image2 = cv2.imread('./data/images/IMG_20241223_150031.jpg')
# Convert the training image to RGB
training_image = cv2.cvtColor(image1, cv2.COLOR_BGR2RGB)
test_image = cv2.cvtColor(image2, cv2.COLOR_BGR2RGB)
# Convert the training image to gray scale
training_gray = cv2.cvtColor(training_image, cv2.COLOR_RGB2GRAY)

# Create test image by adding Scale Invariance and Rotational Invariance
#test_image = training_image
#test_image = cv2.pyrDown(training_image)
#test_image = cv2.pyrDown(test_image)

num_rows, num_cols = test_image.shape[:2]

#rotation_matrix = cv2.getRotationMatrix2D((num_cols/2, num_rows/2), 30, 1)
#test_image = cv2.warpAffine(test_image, rotation_matrix, (num_cols, num_rows))

test_gray = cv2.cvtColor(test_image, cv2.COLOR_RGB2GRAY)

# Display traning image and testing image
fx, plots = plt.subplots(1, 2, figsize=(20,10))

plots[0].set_title("Training Image")
plots[0].imshow(training_image)

plots[1].set_title("Testing Image")
plots[1].imshow(test_image)

In [ ]:
sift = cv2.SIFT_create(500)

train_keypoints, train_descriptor = sift.detectAndCompute(training_gray, None)
test_keypoints, test_descriptor = sift.detectAndCompute(test_gray, None)

keypoints_without_size = np.copy(training_image)
keypoints_with_size = np.copy(training_image)

cv2.drawKeypoints(training_image, train_keypoints, keypoints_without_size, color = (0, 255, 0))

cv2.drawKeypoints(training_image, train_keypoints, keypoints_with_size, flags = cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)

# Display image with and without keypoints size
fx, plots = plt.subplots(1, 2, figsize=(20,10))

plots[0].set_title("Train keypoints With Size")
plots[0].imshow(keypoints_with_size, cmap='gray')

plots[1].set_title("Train keypoints Without Size")
plots[1].imshow(keypoints_without_size, cmap='gray')

# Print the number of keypoints detected in the training image
print("Number of Keypoints Detected In The Training Image: ", len(train_keypoints))

# Print the number of keypoints detected in the query image
print("Number of Keypoints Detected In The Test Image: ", len(test_keypoints))

In [ ]:
keypoints_without_size2 = np.copy(test_image)
keypoints_with_size2 = np.copy(test_image)

cv2.drawKeypoints(test_image, test_keypoints, keypoints_without_size2, color = (0, 255, 0))

cv2.drawKeypoints(test_image, test_keypoints, keypoints_with_size2, flags = cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)

# Display image with and without keypoints size
fx, plots = plt.subplots(1, 2, figsize=(20,10))

plots[0].set_title("Test keypoints With Size")
plots[0].imshow(keypoints_with_size2, cmap='gray')

plots[1].set_title("Test keypoints Without Size")
plots[1].imshow(keypoints_without_size2, cmap='gray')


In [ ]:
# Create a Brute Force Matcher object.
bf = cv2.BFMatcher(cv2.NORM_L1, crossCheck = True)

# Perform the matching between the ORB descriptors of the training image and the test image
matches = bf.match(train_descriptor, test_descriptor)

# The matches with shorter distance are the ones we want.
matches = sorted(matches, key = lambda x : x.distance)
matches = matches[0:]
result = cv2.drawMatches(training_image, train_keypoints, test_image, test_keypoints, matches, test_gray, flags = 2)

# Display the best matching points
plt.rcParams['figure.figsize'] = [30.0, 20.0]
plt.title('Best Matching Points')
plt.imshow(result)
plt.show()

# Print total number of matching points between the training and query images
print("\nNumber of Matching Keypoints Between The Training and Query Images: ", len(matches))

In [ ]:
FLANN_INDEX_KDTREE = 0
index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
search_params = dict(checks=50)
flann = cv2.FlannBasedMatcher(index_params,search_params)
matches = flann.knnMatch(train_descriptor,test_descriptor,k=2)
matchesMask = [[0,0] for i in range(len(matches))]
good = []
for m,n in matches:
    if m.distance < 0.7*n.distance:
        good.append([m])
result2 = cv2.drawMatchesKnn(training_image, train_keypoints, test_image, test_keypoints,good,None,flags=2)
plt.rcParams['figure.figsize'] = [30.0, 20.0]
plt.imshow(result2)
plt.title('Best Matching Points')
plt.show()
print("\nNumber of Matching Keypoints Between The Training and Query Images: ", len(good))


In [ ]:
def load_images_from_folder(folder):
    images = []
    path=(folder + "/")
    for image in os.listdir(path):
        print(path + "/" + image)
        img = cv2.imread(folder + "/" + image)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        #img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
        
        images.append(img)
    return images


In [ ]:
bag_train_images = load_images_from_folder('./data/images')
bag_test_images = load_images_from_folder('./data/test')

In [ ]:
bag_train_descriptors = []
bag_train_keypoints = []
for image in bag_train_images:

    image = cv2.cvtColor(image,cv2.COLOR_RGB2GRAY)
    bag_train_keypoint, bag_train_descriptor = sift.detectAndCompute(image , None)
    bag_train_keypoints.append(bag_train_keypoint)
    bag_train_descriptors.append(bag_train_descriptor)
   


In [ ]:
bag_train_descriptors_array =[]
for des in bag_train_descriptors:
    bag_train_descriptors_array.append(np.array(des).astype(np.float32))

In [ ]:
result=cv2.drawKeypoints(bag_train_images[1], bag_train_keypoints[1], 0, flags = cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
plt.imshow(result)
plt.show()


In [ ]:
all_bag_train_descriptors=[]

for descriptors in bag_train_descriptors_array:
    for descriptor in descriptors:
        all_bag_train_descriptors.append(descriptor)

all_bag_train_descriptors = np.stack(all_bag_train_descriptors)

In [ ]:
all_bag_train_descriptors.shape

In [ ]:
k=400
iters=1
codebook,variance = kmeans(all_bag_train_descriptors,k,iters)

In [ ]:
bag_train_descriptors = []
for image in bag_train_images:

    image = cv2.cvtColor(image,cv2.COLOR_RGB2GRAY)
    bag_train_keypoints, bag_train_descriptor = sift.detectAndCompute(image , None)
    bag_train_descriptors.append(bag_train_descriptor)

In [ ]:
visual_words = []

for descriptor in bag_train_descriptors:
    img_visual_words, distance = vq(descriptor, codebook)
    visual_words.append(img_visual_words)

In [ ]:
print(visual_words[0][:5])
len(visual_words[0])

In [ ]:
frequency_vectors = []
for img_visual_words in visual_words:
    # create a frequency vector for each image
    img_frequency_vector = np.zeros(k)
    for word in img_visual_words:
        img_frequency_vector[word] += 1
    frequency_vectors.append(img_frequency_vector)
# stack together in numpy array
frequency_vectors = np.stack(frequency_vectors)

In [ ]:
frequency_vectors.shape

In [ ]:
for i in [84,  22,  45, 172]:
    print(f"{i}: {frequency_vectors[0][i]}")

In [ ]:
frequency_vectors[0][:20]

In [ ]:
# visualize the frequency vector for image 0
plt.bar(list(range(k)), frequency_vectors[0])
plt.show()

In [ ]:
n=33
df = np.sum(frequency_vectors > 0, axis=0)
df.shape
df[:5]

In [ ]:
idf = np.log(n/ df)
idf.shape
idf[:5]

In [ ]:
tfidf = frequency_vectors * idf
tfidf.shape, tfidf[0][:5]

In [ ]:
plt.bar(list(range(k)), tfidf[0])
plt.show()

In [ ]:
from numpy.linalg import norm

top_k = 5
i = 16

# get search image vector
a = tfidf[i]
b = tfidf  # set search space to the full sample
# get the cosine distance for the search image `a`
cosine_similarity = np.dot(a, b.T)/(norm(a) * norm(b, axis=1))
# get the top k indices for most similar vecs
idx = np.argsort(-cosine_similarity)[:top_k]
# display the results
for i in idx:
    print(f"{i}: {round(cosine_similarity[i], 4)}")
    plt.imshow(bag_train_images[i], cmap='gray')
    plt.show()